# 作业

更换数据集MSRA和ERNIE-Gram或BERT等预训练模型。

- 数据集：
`train_ds, test_ds = load_dataset("msra_ner", splits=["train", "test"])`
- 模型：
	将`from paddlenlp.transformers import ErnieTokenizer, ErnieForTokenClassification`换成相应的模型。

# 使用PaddleNLP语义预训练模型ERNIE完成快递单信息抽取


**注意**

本项目代码需要使用GPU环境来运行:

<img src="https://ai-studio-static-online.cdn.bcebos.com/767f625548714f03b105b6ccb3aa78df9080e38d329e445380f505ddec6c7042" width="40%" height="40%">
<br>
<br>

命名实体识别是NLP中一项非常基础的任务，是信息提取、问答系统、句法分析、机器翻译等众多NLP任务的重要基础工具。命名实体识别的准确度，决定了下游任务的效果，是NLP中的一个基础问题。在NER任务提供了两种解决方案，一类LSTM/GRU + CRF，通过RNN类的模型来抽取底层文本的信息，而CRF(条件随机场)模型来学习底层Token之间的联系；另外一类是通过预训练模型，例如ERNIE，BERT模型，直接来预测Token的标签信息。

本项目将演示如何使用PaddleNLP语义预训练模型ERNIE完成从快递单中抽取姓名、电话、省、市、区、详细地址等内容，形成结构化信息。辅助物流行业从业者进行有效信息的提取，从而降低客户填单的成本。

在2017年之前，工业界和学术界对文本处理依赖于序列模型[Recurrent Neural Network (RNN)](https://baike.baidu.com/item/%E5%BE%AA%E7%8E%AF%E7%A5%9E%E7%BB%8F%E7%BD%91%E7%BB%9C/23199490?fromtitle=RNN&fromid=5707183&fr=aladdin).

<p align="center">
<img src="http://colah.github.io/posts/2015-09-NN-Types-FP/img/RNN-general.png" width="40%" height="30%"> <br />
</p><br><center>图1：RNN示意图</center></br>

[基于BiGRU+CRF的快递单信息抽取](https://aistudio.baidu.com/aistudio/projectdetail/1317771)项目介绍了如何使用序列模型完成快递单信息抽取任务。
<br>

近年来随着深度学习的发展，模型参数的数量飞速增长。为了训练这些参数，需要更大的数据集来避免过拟合。然而，对于大部分NLP任务来说，构建大规模的标注数据集非常困难（成本过高），特别是对于句法和语义相关的任务。相比之下，大规模的未标注语料库的构建则相对容易。为了利用这些数据，我们可以先从其中学习到一个好的表示，再将这些表示应用到其他任务中。最近的研究表明，基于大规模未标注语料库的预训练模型（Pretrained Models, PTM) 在NLP任务上取得了很好的表现。

近年来，大量的研究表明基于大型语料库的预训练模型（Pretrained Models, PTM）可以学习通用的语言表示，有利于下游NLP任务，同时能够避免从零开始训练模型。随着计算能力的不断提高，深度模型的出现（即 Transformer）和训练技巧的增强使得 PTM 不断发展，由浅变深。


<p align="center">
<img src="https://ai-studio-static-online.cdn.bcebos.com/327f44ff3ed24493adca5ddc4dc24bf61eebe67c84a6492f872406f464fde91e" width="60%" height="50%"> <br />
</p><br><center>图2：预训练模型一览，图片来源于：https://github.com/thunlp/PLMpapers</center></br>
                                                                                                                             
本示例展示了以ERNIE([Enhanced Representation through Knowledge Integration](https://arxiv.org/pdf/1904.09223))为代表的预训练模型如何Finetune完成序列标注任务。

**记得给[PaddleNLP](https://github.com/PaddlePaddle/PaddleNLP)点个小小的Star⭐**

开源不易，希望大家多多支持~ 

GitHub地址：[https://github.com/PaddlePaddle/PaddleNLP](https://github.com/PaddlePaddle/PaddleNLP)
![](https://ai-studio-static-online.cdn.bcebos.com/a0e8ca7743ea4fe9aa741682a63e767f8c48dc55981f4e44a40e0e00d3ab369e)

AI Studio平台后续会默认安装PaddleNLP，在此之前可使用如下命令安装。

In [ ]:
!pip install --upgrade paddlenlp

Looking in indexes: https://mirror.baidu.com/pypi/simple/
     |████████████████████████████████| 430kB 14.8MB/s eta 0:00:01
  Found existing installation: paddlenlp 2.0.1
    Uninstalling paddlenlp-2.0.1:
      Successfully uninstalled paddlenlp-2.0.1


In [ ]:
# 下载并解压数据集
# from paddle.utils.download import get_path_from_url
# # URL = "https://paddlenlp.bj.bcebos.com/paddlenlp/datasets/msra_ner.tar.gz"
# URL = 'https://paddlenlp.bj.bcebos.com/datasets/msra_ner.tar.gz'
# get_path_from_url(URL, "./")

# # 查看预测的数据
# !head -n 5 msra_ner/test.tsv

中共中央致中国致公党十一大的贺词各位代表、各位同志：在中国致公党第十一次全国代表大会隆重召开之际，中国共产党中央委员会谨向大会表示热烈的祝贺，向致公党的同志们致以亲切的问候！	B-ORGI-ORGI-ORGI-ORGOB-ORGI-ORGI-ORGI-ORGI-ORGI-ORGI-ORGI-ORGOOOOOOOOOOOOOOB-ORGI-ORGI-ORGI-ORGI-ORGI-ORGI-ORGI-ORGI-ORGI-ORGI-ORGI-ORGI-ORGI-ORGI-ORGOOOOOOOB-ORGI-ORGI-ORGI-ORGI-ORGI-ORGI-ORGI-ORGI-ORGI-ORGOOOOOOOOOOOOOB-ORGI-ORGI-ORGOOOOOOOOOOOO
在过去的五年中，致公党在邓小平理论指引下，遵循社会主义初级阶段的基本路线，努力实践致公党十大提出的发挥参政党职能、加强自身建设的基本任务。	OOOOOOOOB-ORGI-ORGI-ORGOB-PERI-PERI-PEROOOOOOOOOOOOOOOOOOOOOOOOOOB-ORGI-ORGI-ORGI-ORGI-ORGOOOOOOOOOOOOOOOOOOOOOOO
高举爱国主义和社会主义两面旗帜，团结全体成员以及所联系的归侨、侨眷，发扬爱国革命的光荣传统，为统一祖国、振兴中华而努力奋斗；紧紧围绕国家的中心工作，联系改革和建设中的重大问

In [12]:
from functools import partial
import time
import paddle
from paddlenlp.datasets import MapDataset
from paddlenlp.data import Stack, Tuple, Pad, Dict
# from paddlenlp.transformers import ErnieTokenizer, ErnieForTokenClassification
from paddlenlp.transformers import BertForTokenClassification, BertTokenizer
from paddlenlp.metrics import ChunkEvaluator
from utils import convert_example, evaluate, predict, load_dict
from paddlenlp.datasets import load_dataset
from paddle.io import DataLoader
from paddlenlp.transformers import LinearDecayWithWarmup

In [13]:
train_ds, test_ds = load_dataset(
        'msra_ner', splits=('train', 'test'), lazy=False)

In [14]:
tokenizer = BertTokenizer.from_pretrained('bert-wwm-chinese')

[2021-06-16 13:27:39,673] [    INFO] - Found /home/aistudio/.paddlenlp/models/bert-wwm-chinese/bert-wwm-chinese-vocab.txt


In [15]:
def evaluate(model, loss_fct, metric, data_loader, label_num):
    model.eval()
    metric.reset()
    avg_loss, precision, recall, f1_score = 0, 0, 0, 0
    for batch in data_loader:
        input_ids, token_type_ids, length, labels = batch
        logits = model(input_ids, token_type_ids)
        loss = loss_fct(logits, labels)
        avg_loss = paddle.mean(loss)
        preds = logits.argmax(axis=2)
        num_infer_chunks, num_label_chunks, num_correct_chunks = metric.compute(
            length, preds, labels)
        metric.update(num_infer_chunks.numpy(),
                      num_label_chunks.numpy(), num_correct_chunks.numpy())
        precision, recall, f1_score = metric.accumulate()
    print("eval loss: %f, precision: %f, recall: %f, f1: %f" %
          (avg_loss, precision, recall, f1_score))
    model.train()

In [16]:
def tokenize_and_align_labels(example, tokenizer, no_entity_id,
                              max_seq_len=512):
    labels = example['labels']
    example = example['tokens']
    tokenized_input = tokenizer(
        example,
        return_length=True,
        is_split_into_words=True,
        max_seq_len=max_seq_len)

    # -2 for [CLS] and [SEP]
    if len(tokenized_input['input_ids']) - 2 < len(labels):
        labels = labels[:len(tokenized_input['input_ids']) - 2]
    tokenized_input['labels'] = [no_entity_id] + labels + [no_entity_id]
    tokenized_input['labels'] += [no_entity_id] * (
        len(tokenized_input['input_ids']) - len(tokenized_input['labels']))
    return tokenized_input

In [17]:
label_list = train_ds.label_list
label_num = len(label_list)
no_entity_id = label_num - 1

trans_func = partial(
        tokenize_and_align_labels,
        tokenizer=tokenizer,
        no_entity_id=no_entity_id,
        max_seq_len=128)

train_ds = train_ds.map(trans_func)

ignore_label = -100

batchify_fn = lambda samples, fn=Dict({
        'input_ids': Pad(axis=0, pad_val=tokenizer.pad_token_id, dtype='int32'),  # input
        'token_type_ids': Pad(axis=0, pad_val=tokenizer.pad_token_type_id, dtype='int32'),  # segment
        'seq_len': Stack(dtype='int64'),  # seq_len
        'labels': Pad(axis=0, pad_val=ignore_label, dtype='int64')  # label
    }): fn(samples)

train_batch_sampler = paddle.io.DistributedBatchSampler(
        train_ds, batch_size=8, shuffle=True, drop_last=True)

train_data_loader = DataLoader(
        dataset=train_ds,
        collate_fn=batchify_fn,
        num_workers=0,
        batch_sampler=train_batch_sampler,
        return_list=True)

test_ds = test_ds.map(trans_func)

test_data_loader = DataLoader(
        dataset=test_ds,
        collate_fn=batchify_fn,
        num_workers=0,
        batch_size=8,
        return_list=True)

    # Define the model netword and its loss
model = BertForTokenClassification.from_pretrained(
        'bert-wwm-chinese', num_classes=label_num)
if paddle.distributed.get_world_size() > 1:
        model = paddle.DataParallel(model)

num_training_steps = -1 if -1 > 0 else len(
        train_data_loader) * 3

lr_scheduler = LinearDecayWithWarmup(5e-5, num_training_steps,0)

    # Generate parameter names needed to perform weight decay.
    # All bias and LayerNorm parameters are excluded.
decay_params = [
        p.name for n, p in model.named_parameters()
        if not any(nd in n for nd in ["bias", "norm"])
    ]
optimizer = paddle.optimizer.AdamW(
        learning_rate=lr_scheduler,
        epsilon=1e-8,
        parameters=model.parameters(),
        weight_decay=0.0,
        apply_decay_param_fun=lambda x: x in decay_params)

loss_fct = paddle.nn.loss.CrossEntropyLoss(ignore_index=ignore_label)

metric = ChunkEvaluator(label_list=label_list)

global_step = 0
last_step = 3 * len(train_data_loader)
tic_train = time.time()
for epoch in range(3):
    for step, batch in enumerate(train_data_loader):
        global_step += 1
        input_ids, token_type_ids, _, labels = batch
        logits = model(input_ids, token_type_ids)
        loss = loss_fct(logits, labels)
        avg_loss = paddle.mean(loss)
        if global_step % 1 == 0:
            print(
                    "global step %d, epoch: %d, batch: %d, loss: %f, speed: %.2f step/s"
                    % (global_step, epoch, step, avg_loss,
                       1 / (time.time() - tic_train)))
            tic_train = time.time()
        avg_loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.clear_grad()
        if global_step % 100 == 0 or global_step == last_step:
            if paddle.distributed.get_rank() == 0:
                evaluate(model, loss_fct, metric, test_data_loader,
                             label_num)
                # paddle.save(model.state_dict(),
                #                 os.path.join(args.output_dir,
                #                              "model_%d.pdparams" % global_step))
                paddle.save(model.state_dict(),
                './ernie_result/model_%d.pdparams' % step)

[2021-06-16 13:27:42,824] [    INFO] - Already cached /home/aistudio/.paddlenlp/models/bert-wwm-chinese/bert-wwm-chinese.pdparams


global step 1, epoch: 0, batch: 0, loss: 1.730545, speed: 5.53 step/s
global step 2, epoch: 0, batch: 1, loss: 0.624672, speed: 4.34 step/s
global step 3, epoch: 0, batch: 2, loss: 0.512653, speed: 6.45 step/s
global step 4, epoch: 0, batch: 3, loss: 0.272759, speed: 6.88 step/s
global step 5, epoch: 0, batch: 4, loss: 0.740571, speed: 7.49 step/s
global step 6, epoch: 0, batch: 5, loss: 0.377271, speed: 7.99 step/s
global step 7, epoch: 0, batch: 6, loss: 0.300970, speed: 7.04 step/s
global step 8, epoch: 0, batch: 7, loss: 0.273996, speed: 7.72 step/s
global step 9, epoch: 0, batch: 8, loss: 0.136995, speed: 7.08 step/s
global step 10, epoch: 0, batch: 9, loss: 0.241111, speed: 7.38 step/s
global step 11, epoch: 0, batch: 10, loss: 0.049911, speed: 9.15 step/s
global step 12, epoch: 0, batch: 11, loss: 0.252745, speed: 6.99 step/s
global step 13, epoch: 0, batch: 12, loss: 0.455495, speed: 7.83 step/s
global step 14, epoch: 0, batch: 13, loss: 0.265735, speed: 9.06 step/s
global step

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/tensor/creation.py:125: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if data.dtype == np.object:


In [31]:
# preds = predict(model, test_data_loader, test_ds, label_num)
# file_path = "ernie_results.txt"
# with open(file_path, "w", encoding="utf8") as fout:
#     fout.write("\n".join(preds))
# # Print some examples
# print(
#     "The results have been saved in the file: %s, some examples are shown below: "
#     % file_path)
# print("\n".join(preds[:10]))
def parse_decodes(input_words, id2label, decodes, lens):
    decodes = [x for batch in decodes for x in batch]
    lens = [x for batch in lens for x in batch]

    outputs = []
    for idx, end in enumerate(lens):
        sent = "".join(input_words[idx]['tokens'])
        tags = [id2label[x] for x in decodes[idx][1:end]]
        sent_out = []
        tags_out = []
        words = ""
        for s, t in zip(sent, tags):
            if t.startswith('B-') or t == 'O':
                if len(words):
                    sent_out.append(words)
                if t.startswith('B-'):
                    tags_out.append(t.split('-')[1])
                else:
                    tags_out.append(t)
                words = s
            else:
                words += s
        if len(sent_out) < len(tags_out):
            sent_out.append(words)
        outputs.append(''.join(
            [str((s, t)) for s, t in zip(sent_out, tags_out)]))
    return outputs
train_ds, predict_ds = load_dataset(
        'msra_ner', splits=('train', 'test'), lazy=False)
raw_data = predict_ds.data
id2label = dict(enumerate(predict_ds.label_list))

predict_ds = predict_ds.map(trans_func)
predict_data_loader = DataLoader(
    dataset=predict_ds,
    collate_fn=batchify_fn,
    num_workers=0,
    batch_size=8,
    return_list=True)

pred_list = []
len_list = []
for step, batch in enumerate(predict_data_loader):
    input_ids, token_type_ids, length, labels = batch
    logits = model(input_ids, token_type_ids)
    pred = paddle.argmax(logits, axis=-1)
    pred_list.append(pred.numpy())
    len_list.append(length.numpy())
preds = parse_decodes(raw_data, id2label, pred_list, len_list)

file_path = "ernie_results.txt"
with open(file_path, "w", encoding="utf8") as fout:
        fout.write("\n".join(preds))
    # Print some examples
print(
        "The results have been saved in the file: %s, some examples are shown below: "
        % file_path)
print("\n".join(preds[:10]))


The results have been saved in the file: ernie_results.txt, some examples are shown below: 
('中共中央', 'ORG')('致', 'O')('中国致公党十一大', 'ORG')('的', 'O')('贺', 'O')('词', 'O')('各', 'O')('位', 'O')('代', 'O')('表', 'O')('、', 'O')('各', 'O')('位', 'O')('同', 'O')('志', 'O')('：', 'O')('在', 'O')('中国致公党第十一次全国代表大会', 'ORG')('隆', 'O')('重', 'O')('召', 'O')('开', 'O')('之', 'O')('际', 'O')('，', 'O')('中国共产党中央委员会', 'ORG')('谨', 'O')('向', 'O')('大', 'O')('会', 'O')('表', 'O')('示', 'O')('热', 'O')('烈', 'O')('的', 'O')('祝', 'O')('贺', 'O')('，', 'O')('向', 'O')('致公党', 'ORG')('的', 'O')('同', 'O')('志', 'O')('们', 'O')('致', 'O')('以', 'O')('亲', 'O')('切', 'O')('的', 'O')('问', 'O')('候', 'O')('！', 'O')
('在', 'O')('过', 'O')('去', 'O')('的', 'O')('五', 'O')('年', 'O')('中', 'O')('，', 'O')('致公党', 'ORG')('在', 'O')('邓小平', 'PER')('理', 'O')('论', 'O')('指', 'O')('引', 'O')('下', 'O')('，', 'O')('遵', 'O')('循', 'O')('社', 'O')('会', 'O')('主', 'O')('义', 'O')('初', 'O')('级', 'O')('阶', 'O')('段', 'O')('的', 'O')('基', 'O')('本', 'O')('路', 'O')('线', 'O')('，', 'O')('努'

## 加载自定义数据集

推荐使用MapDataset()自定义数据集。

In [32]:
# 下载并解压数据集
from paddle.utils.download import get_path_from_url
URL = "https://paddlenlp.bj.bcebos.com/paddlenlp/datasets/waybill.tar.gz"
get_path_from_url(URL, "./")

# 查看预测的数据
!head -n 5 data/test.txt

text_a	label
黑龙江省双鸭山市尖山区八马路与东平行路交叉口北40米韦业涛18600009172	A1-BA1-IA1-IA1-IA2-BA2-IA2-IA2-IA3-BA3-IA3-IA4-BA4-IA4-IA4-IA4-IA4-IA4-IA4-IA4-IA4-IA4-IA4-IA4-IA4-IA4-IP-BP-IP-IT-BT-IT-IT-IT-IT-IT-IT-IT-IT-IT-I
广西壮族自治区桂林市雁山区雁山镇西龙村老年活动中心17610348888羊卓卫	A1-BA1-IA1-IA1-IA1-IA1-IA1-IA2-BA2-IA2-IA3-BA3-IA3-IA4-BA4-IA4-IA4-IA4-IA4-IA4-IA4-IA4-IA4-IA4-IA4-IT-BT-IT-IT-IT-IT-IT-IT-IT-IT-IT-IP-BP-IP-I
15652864561河南省开封市顺河回族区顺河区公园路32号赵本山	T-BT-IT-IT-IT-IT-IT-IT-IT-IT-IT-IA1-BA1-IA1-IA2-BA2-IA2-IA3-BA3-IA3-IA3-IA3-IA4-BA4-IA4-IA4-IA4-IA4-IA4-IA4-IA4-IP-BP-IP-I
河北省唐山市玉田县无终大街159号18614253058尚汉生	A1-BA1-IA1-IA2-BA2-IA2-IA3-BA3-IA3-IA4-BA4-IA4-IA4-IA4-IA4-IA4-IA4-IT-BT-IT-IT-IT-IT-IT-IT-IT-IT-IT-IP-BP-IP-I


In [33]:

from functools import partial

import paddle
from paddlenlp.datasets import MapDataset
from paddlenlp.data import Stack, Tuple, Pad
from paddlenlp.transformers import ErnieTokenizer, ErnieForTokenClassification
from paddlenlp.metrics import ChunkEvaluator
from utils import convert_example, evaluate, predict, load_dict

In [34]:
def load_dataset(datafiles):
    def read(data_path):
        with open(data_path, 'r', encoding='utf-8') as fp:
            next(fp)  # Skip header
            for line in fp.readlines():
                words, labels = line.strip('\n').split('\t')
                words = words.split('\002')
                labels = labels.split('\002')
                yield words, labels

    if isinstance(datafiles, str):
        return MapDataset(list(read(datafiles)))
    elif isinstance(datafiles, list) or isinstance(datafiles, tuple):
        return [MapDataset(list(read(datafile))) for datafile in datafiles]

# Create dataset, tokenizer and dataloader.
train_ds, dev_ds, test_ds = load_dataset(datafiles=(
        './data/train.txt', './data/dev.txt', './data/test.txt'))

In [35]:
# def load_dataset(datafiles):
#     def read(data_path):
#         with open(data_path, 'r', encoding='utf-8') as fp:
#             next(fp)  # Skip header
#             for line in fp.readlines():
#                 words, labels = line.strip('\n').split('\t')
#                 words = words.split('\002')
#                 labels = labels.split('\002')
#                 yield words, labels

#     if isinstance(datafiles, str):
#         return MapDataset(list(read(datafiles)))
#     elif isinstance(datafiles, list) or isinstance(datafiles, tuple):
#         return [MapDataset(list(read(datafile))) for datafile in datafiles]

# # Create dataset, tokenizer and dataloader.
# train_ds, test_ds = load_dataset(datafiles=(
#         './msra_ner/train.tsv', './msra_ner/test.tsv'))

In [36]:
for i in range(5):
    print(train_ds[i])

(['1', '6', '6', '2', '0', '2', '0', '0', '0', '7', '7', '宣', '荣', '嗣', '甘', '肃', '省', '白', '银', '市', '会', '宁', '县', '河', '畔', '镇', '十', '字', '街', '金', '海', '超', '市', '西', '行', '5', '0', '米'], ['T-B', 'T-I', 'T-I', 'T-I', 'T-I', 'T-I', 'T-I', 'T-I', 'T-I', 'T-I', 'T-I', 'P-B', 'P-I', 'P-I', 'A1-B', 'A1-I', 'A1-I', 'A2-B', 'A2-I', 'A2-I', 'A3-B', 'A3-I', 'A3-I', 'A4-B', 'A4-I', 'A4-I', 'A4-I', 'A4-I', 'A4-I', 'A4-I', 'A4-I', 'A4-I', 'A4-I', 'A4-I', 'A4-I', 'A4-I', 'A4-I', 'A4-I'])
(['1', '3', '5', '5', '2', '6', '6', '4', '3', '0', '7', '姜', '骏', '炜', '云', '南', '省', '德', '宏', '傣', '族', '景', '颇', '族', '自', '治', '州', '盈', '江', '县', '平', '原', '镇', '蜜', '回', '路', '下', '段'], ['T-B', 'T-I', 'T-I', 'T-I', 'T-I', 'T-I', 'T-I', 'T-I', 'T-I', 'T-I', 'T-I', 'P-B', 'P-I', 'P-I', 'A1-B', 'A1-I', 'A1-I', 'A2-B', 'A2-I', 'A2-I', 'A2-I', 'A2-I', 'A2-I', 'A2-I', 'A2-I', 'A2-I', 'A2-I', 'A3-B', 'A3-I', 'A3-I', 'A4-B', 'A4-I', 'A4-I', 'A4-I', 'A4-I', 'A4-I', 'A4-I', 'A4-I'])
(['内', '蒙', '古', '自', '治', '区'

每条数据包含一句文本和这个文本中每个汉字以及数字对应的label标签。

之后，还需要对输入句子进行数据处理，如切词，映射词表id等。

## 数据处理

预训练模型ERNIE对中文数据的处理是以字为单位。PaddleNLP对于各种预训练模型已经内置了相应的tokenizer。指定想要使用的模型名字即可加载对应的tokenizer。

tokenizer作用为将原始输入文本转化成模型model可以接受的输入数据形式。


<p align="center">
<img src="https://bj.bcebos.com/paddlehub/paddlehub-img/ernie_network_1.png" hspace='10'/> <br />
</p>


<p align="center">
<img src="https://bj.bcebos.com/paddlehub/paddlehub-img/ernie_network_2.png" hspace='10'/> <br />
</p>
<br><center>图3：ERNIE模型示意图</center></br>

In [37]:
# label_vocab = load_dict('./msra_ner/label_map.json')
# # tokenizer = ErnieTokenizer.from_pretrained('ernie-1.0')
# tokenizer = BertTokenizer.from_pretrained("bert-wwm-chinese")
# trans_func = partial(convert_example, tokenizer=tokenizer, label_vocab=label_vocab)

# train_ds.map(trans_func)
# # dev_ds.map(trans_func)
# test_ds.map(trans_func)


In [38]:
label_vocab = load_dict('./data/tag.dic')
tokenizer = ErnieTokenizer.from_pretrained('ernie-1.0')

trans_func = partial(convert_example, tokenizer=tokenizer, label_vocab=label_vocab)

train_ds.map(trans_func)
dev_ds.map(trans_func)
test_ds.map(trans_func)
print (train_ds[0])

[2021-06-16 16:37:33,955] [    INFO] - Downloading vocab.txt from https://paddlenlp.bj.bcebos.com/models/transformers/ernie/vocab.txt
100%|██████████| 90/90 [00:00<00:00, 3559.56it/s]


([1, 208, 515, 515, 249, 540, 249, 540, 540, 540, 589, 589, 803, 838, 2914, 1222, 1734, 244, 368, 797, 99, 32, 863, 308, 457, 2778, 484, 167, 436, 930, 192, 233, 634, 99, 213, 40, 317, 540, 256, 2], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 40, [12, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 1, 1, 4, 5, 5, 6, 7, 7, 8, 9, 9, 10, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 12])


### 数据读入

使用`paddle.io.DataLoader`接口多线程异步加载数据。

In [39]:
ignore_label = -1
batchify_fn = lambda samples, fn=Tuple(
    Pad(axis=0, pad_val=tokenizer.pad_token_id),  # input_ids
    Pad(axis=0, pad_val=tokenizer.pad_token_type_id),  # token_type_ids
    Stack(),  # seq_len
    Pad(axis=0, pad_val=ignore_label)  # labels
): fn(samples)

train_loader = paddle.io.DataLoader(
    dataset=train_ds,
    batch_size=36,
    return_list=True,
    collate_fn=batchify_fn)
dev_loader = paddle.io.DataLoader(
    dataset=dev_ds,
    batch_size=36,
    return_list=True,
    collate_fn=batchify_fn)
test_loader = paddle.io.DataLoader(
    dataset=test_ds,
    batch_size=36,
    return_list=True,
    collate_fn=batchify_fn)

## PaddleNLP一键加载预训练模型


快递单信息抽取本质是一个序列标注任务，PaddleNLP对于各种预训练模型已经内置了对于下游任务文本分类Fine-tune网络。以下教程以ERNIE为预训练模型完成序列标注任务。

`paddlenlp.transformers.ErnieForTokenClassification()`一行代码即可加载预训练模型ERNIE用于序列标注任务的fine-tune网络。其在ERNIE模型后拼接上一个全连接网络进行分类。

`paddlenlp.transformers.ErnieForTokenClassification.from_pretrained()`方法只需指定想要使用的模型名称和文本分类的类别数即可完成定义模型网络。

In [40]:
# Define the model netword and its loss
model = ErnieForTokenClassification.from_pretrained("ernie-1.0", num_classes=len(label_vocab))
# model = BertForSequenceClassification.from_pretrained("bert-wwm-chinese", num_classes=label_num)

[2021-06-16 16:37:36,395] [    INFO] - Downloading https://paddlenlp.bj.bcebos.com/models/transformers/ernie/ernie_v1_chn_base.pdparams and saved to /home/aistudio/.paddlenlp/models/ernie-1.0
[2021-06-16 16:37:36,397] [    INFO] - Downloading ernie_v1_chn_base.pdparams from https://paddlenlp.bj.bcebos.com/models/transformers/ernie/ernie_v1_chn_base.pdparams
100%|██████████| 392507/392507 [00:07<00:00, 53042.13it/s]


PaddleNLP不仅支持ERNIE预训练模型，还支持BERT、RoBERTa、Electra等预训练模型。
下表汇总了目前PaddleNLP支持的各类预训练模型。您可以使用PaddleNLP提供的模型，完成文本分类、序列标注、问答等任务。同时我们提供了众多预训练模型的参数权重供用户使用，其中包含了二十多种中文语言模型的预训练权重。中文的预训练模型有`bert-base-chinese, bert-wwm-chinese, bert-wwm-ext-chinese, ernie-1.0, ernie-tiny, gpt2-base-cn, roberta-wwm-ext, roberta-wwm-ext-large, rbt3, rbtl3, chinese-electra-base, chinese-electra-small, chinese-xlnet-base, chinese-xlnet-mid, chinese-xlnet-large, unified_transformer-12L-cn, unified_transformer-12L-cn-luge`等。

更多预训练模型参考：[PaddleNLP Transformer API](https://github.com/PaddlePaddle/PaddleNLP/blob/develop/docs/transformers.md)。

更多预训练模型fine-tune下游任务使用方法，请参考：[examples](https://github.com/PaddlePaddle/PaddleNLP/tree/develop/examples)。

## 设置Fine-Tune优化策略，模型配置
适用于ERNIE/BERT这类Transformer模型的迁移优化学习率策略为warmup的动态学习率。

<p align="center">
<img src="https://ai-studio-static-online.cdn.bcebos.com/2bc624280a614a80b5449773192be460f195b13af89e4e5cbaf62bf6ac16de2c" width="40%" height="30%"/> <br />
</p><br><center>图4：动态学习率示意图</center></br>



In [41]:
metric = ChunkEvaluator(label_list=label_vocab.keys(), suffix=True)
loss_fn = paddle.nn.loss.CrossEntropyLoss(ignore_index=ignore_label)
optimizer = paddle.optimizer.AdamW(learning_rate=2e-5, parameters=model.parameters())

## 模型训练与评估


模型训练的过程通常有以下步骤：

1. 从dataloader中取出一个batch data
2. 将batch data喂给model，做前向计算
3. 将前向计算结果传给损失函数，计算loss。将前向计算结果传给评价方法，计算评价指标。
4. loss反向回传，更新梯度。重复以上步骤。

每训练一个epoch时，程序将会评估一次，评估当前模型训练的效果。

In [42]:
step = 0
for epoch in range(10):
    for idx, (input_ids, token_type_ids, length, labels) in enumerate(train_loader):
        logits = model(input_ids, token_type_ids)
        loss = paddle.mean(loss_fn(logits, labels))
        loss.backward()
        optimizer.step()
        optimizer.clear_grad()
        step += 1
        print("epoch:%d - step:%d - loss: %f" % (epoch, step, loss))
    evaluate(model, metric, dev_loader)

    paddle.save(model.state_dict(),
                './ernie_result/model_%d.pdparams' % step)
# model.save_pretrained('./checkpoint')
# tokenizer.save_pretrained('./checkpoint')

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dygraph/math_op_patch.py:239: UserWarning: The dtype of left and right variables are not the same, left dtype is paddle.float32, but right dtype is paddle.bool, the right dtype will convert to paddle.float32
  format(lhs_dtype, rhs_dtype, lhs_dtype))


epoch:0 - step:1 - loss: 2.641742
epoch:0 - step:2 - loss: 2.421458
epoch:0 - step:3 - loss: 2.241805
epoch:0 - step:4 - loss: 2.076534
epoch:0 - step:5 - loss: 1.952870
epoch:0 - step:6 - loss: 1.851481
epoch:0 - step:7 - loss: 1.726657
epoch:0 - step:8 - loss: 1.603177
epoch:0 - step:9 - loss: 1.530140
epoch:0 - step:10 - loss: 1.424838
epoch:0 - step:11 - loss: 1.365694
epoch:0 - step:12 - loss: 1.310175
epoch:0 - step:13 - loss: 1.259889
epoch:0 - step:14 - loss: 1.191792
epoch:0 - step:15 - loss: 1.168801
epoch:0 - step:16 - loss: 1.084729
epoch:0 - step:17 - loss: 1.093034
epoch:0 - step:18 - loss: 1.012313
epoch:0 - step:19 - loss: 0.970749
epoch:0 - step:20 - loss: 0.924451
epoch:0 - step:21 - loss: 0.869113
epoch:0 - step:22 - loss: 0.841398
epoch:0 - step:23 - loss: 0.804191
epoch:0 - step:24 - loss: 0.776479
epoch:0 - step:25 - loss: 0.769694
epoch:0 - step:26 - loss: 0.716212
epoch:0 - step:27 - loss: 0.713279
epoch:0 - step:28 - loss: 0.685468
epoch:0 - step:29 - loss: 0.6

[2021-06-16 16:38:15,477] [ WARNING] - Compatibility Warning: The params of ChunkEvaluator.compute has been modified. The old version is `inputs`, `lengths`, `predictions`, `labels` while the current version is `lengths`, `predictions`, `labels`.  Please update the usage.


eval precision: 0.932190 - recall: 0.959630 - f1: 0.945711
epoch:1 - step:46 - loss: 0.230952
epoch:1 - step:47 - loss: 0.215577
epoch:1 - step:48 - loss: 0.221225
epoch:1 - step:49 - loss: 0.199531
epoch:1 - step:50 - loss: 0.198224
epoch:1 - step:51 - loss: 0.175371
epoch:1 - step:52 - loss: 0.163069
epoch:1 - step:53 - loss: 0.134598
epoch:1 - step:54 - loss: 0.133064
epoch:1 - step:55 - loss: 0.126076
epoch:1 - step:56 - loss: 0.107437
epoch:1 - step:57 - loss: 0.124397
epoch:1 - step:58 - loss: 0.085090
epoch:1 - step:59 - loss: 0.078551
epoch:1 - step:60 - loss: 0.136946
epoch:1 - step:61 - loss: 0.104542
epoch:1 - step:62 - loss: 0.087848
epoch:1 - step:63 - loss: 0.065619
epoch:1 - step:64 - loss: 0.086226
epoch:1 - step:65 - loss: 0.086361
epoch:1 - step:66 - loss: 0.066396
epoch:1 - step:67 - loss: 0.078656
epoch:1 - step:68 - loss: 0.065434
epoch:1 - step:69 - loss: 0.061247
epoch:1 - step:70 - loss: 0.076129
epoch:1 - step:71 - loss: 0.063647
epoch:1 - step:72 - loss: 0.062

In [ ]:
# import time
# global_step = 0
# last_step = 10 * len(train_data_loader)
# tic_train = time.time()
# for epoch in range(10):
#         for step, batch in enumerate(train_data_loader):
#             global_step += 1
#             input_ids, token_type_ids, _, labels = batch
#             logits = model(input_ids, token_type_ids)
#             loss = loss_fct(logits, labels)
#             avg_loss = paddle.mean(loss)
#             if global_step % args.logging_steps == 0:
#                 print(
#                     "global step %d, epoch: %d, batch: %d, loss: %f, speed: %.2f step/s"
#                     % (global_step, epoch, step, avg_loss,
#                        args.logging_steps / (time.time() - tic_train)))
#                 tic_train = time.time()
#             avg_loss.backward()
#             optimizer.step()
#             lr_scheduler.step()
#             optimizer.clear_grad()
#             if global_step % args.save_steps == 0 or global_step == last_step:
#                 if paddle.distributed.get_rank() == 0:
#                     evaluate(model, loss_fct, metric, test_data_loader,
#                              label_num)
#                     paddle.save(model.state_dict(),
#                                 os.path.join(args.output_dir,
#                                              "model_%d.pdparams" % global_step))

In [ ]:
# criterion = paddle.nn.loss.CrossEntropyLoss()
# step = 0
# for epoch in range(10):
#     for idx, (input_ids, token_type_ids, length, labels) in enumerate(train_loader):
#         logits = model(input_ids, token_type_ids)
#         loss = criterion(logits, labels)
#         probs = paddle.nn.functional.softmax(logits, axis=1)
#         loss.backward()
#         optimizer.step()
#         optimizer.clear_grad()
#         step += 1
#         print("epoch:%d - step:%d - loss: %f" % (epoch, step, loss))
#     evaluate(model, metric, dev_loader)

#     paddle.save(model.state_dict(),
#                 './ernie_result/model_%d.pdparams' % step)
# model.save_pretrained('./checkpoint')
# tokenizer.save_pretrained('./checkpoint')

## 模型预测

训练保存好的模型，即可用于预测。如以下示例代码自定义预测数据，调用`predict()`函数即可一键预测。

In [43]:
preds = predict(model, test_loader, test_ds, label_vocab)
file_path = "ernie_results.txt"
with open(file_path, "w", encoding="utf8") as fout:
    fout.write("\n".join(preds))
# Print some examples
print(
    "The results have been saved in the file: %s, some examples are shown below: "
    % file_path)
print("\n".join(preds[:10]))

The results have been saved in the file: ernie_results.txt, some examples are shown below: 
('黑龙江省', 'A1')('双鸭山市', 'A2')('尖山区', 'A3')('八马路与东平行路交叉口北40米', 'A4')('韦业涛', 'P')('18600009172', 'T')
('广西壮族自治区', 'A1')('桂林市', 'A2')('雁山区', 'A3')('雁山镇西龙村老年活动中心', 'A4')('17610348888', 'T')('羊卓卫', 'P')
('15652864561', 'T')('河南省', 'A1')('开封市', 'A2')('顺河回族区', 'A3')('顺河区公园路32号', 'A4')('赵本山', 'P')
('河北省', 'A1')('唐山市', 'A2')('玉田县', 'A3')('无终大街159号', 'A4')('18614253058', 'T')('尚汉生', 'P')
('台湾', 'A1')('台中市', 'A2')('北区', 'A3')('北区锦新街18号', 'A4')('18511226708', 'T')('蓟丽', 'P')
('廖梓琪', 'P')('18514743222', 'T')('湖北省', 'A1')('宜昌市', 'A2')('长阳土家族自治县', 'A3')('贺家坪镇贺家坪村一组临河1号', 'A4')
('江苏省', 'A1')('南通市', 'A2')('海门市', 'A3')('孝威村孝威路88号', 'A4')('18611840623', 'T')('计星仪', 'P')
('17601674746', 'T')('赵春丽', 'P')('内蒙古自治区', 'A1')('乌兰察布市', 'A2')('凉城县', 'A3')('新建街', 'A4')
('云南省', 'A1')('临沧市', 'A2')('耿马傣族佤族自治县', 'A3')('鑫源路法院对面', 'A4')('许贞爱', 'P')('18510566685', 'T')
('四川省', 'A1')('成都市', 'A2')('双流区', 'A3')('东升镇北仓路196号', 'A4')('耿丕岭

# 进一步使用CRF

PaddleNLP提供了CRF Layer，它能够学习label之间的关系，能够帮助模型更好地学习、预测序列标注任务。

我们在PaddleNLP仓库中提供了[示例](https://github.com/PaddlePaddle/PaddleNLP/blob/develop/examples/information_extraction/waybill_ie/run_ernie_crf.py)，您可以参照示例代码使用Ernie-CRF结构完成快递单信息抽取任务。

In [ ]:
import paddle.nn as nn
from paddlenlp.transformers import ErniePretrainedModel
from paddlenlp.layers.crf import LinearChainCrf, ViterbiDecoder, LinearChainCrfLoss


class ErnieCrfForTokenClassification(nn.Layer):
    def __init__(self, ernie, crf_lr=100):
        super().__init__()
        self.num_classes = ernie.num_classes
        self.ernie = ernie  # allow ernie to be config
        self.crf = LinearChainCrf(
            self.num_classes, crf_lr=crf_lr, with_start_stop_tag=False)
        self.crf_loss = LinearChainCrfLoss(self.crf)
        self.viterbi_decoder = ViterbiDecoder(
            self.crf.transitions, with_start_stop_tag=False)

    def forward(self,
                input_ids,
                token_type_ids=None,
                position_ids=None,
                attention_mask=None,
                lengths=None,
                labels=None):
        logits = self.ernie(
            input_ids,
            token_type_ids=token_type_ids,
            attention_mask=attention_mask,
            position_ids=position_ids)

        if labels is not None:
            loss = self.crf_loss(logits, lengths, labels)
            return loss
        else:
            _, prediction = self.viterbi_decoder(logits, lengths)
            return prediction

In [ ]:
ernie = ErnieForTokenClassification.from_pretrained(
        "ernie-1.0", num_classes=len(label_vocab))
model = ErnieCrfForTokenClassification(ernie)

In [47]:
metric = ChunkEvaluator(label_list=label_vocab.keys(), suffix=True)
optimizer = paddle.optimizer.AdamW(
        learning_rate=2e-5, parameters=model.parameters())

In [48]:
step = 0
for epoch in range(10):
    for input_ids, token_type_ids, lengths, labels in train_loader:
        loss = model(
                input_ids, token_type_ids, lengths=lengths, labels=labels)
        avg_loss = paddle.mean(loss)
        avg_loss.backward()
        optimizer.step()
        optimizer.clear_grad()
        step += 1
        print("[TRAIN] Epoch:%d - Step:%d - Loss: %f" %
                  (epoch, step, avg_loss))
    evaluate(model, metric, dev_loader)

    paddle.save(model.state_dict(),
                    './ernie_crf_ckpt/model_%d.pdparams' % step)

In [ ]:
preds = predict(model, test_loader, test_ds, label_vocab)
# file_path = "ernie_crf_results.txt"
file_path = "ernie_results.txt"
with open(file_path, "w", encoding="utf8") as fout:
    fout.write("\n".join(preds))
    # Print some examples
print("The results have been saved in the file: %s, some examples are shown below: "
        % file_path)
print("\n".join(preds[:10]))

# 加入交流群，一起学习吧

现在就加入课程QQ交流群，一起交流NLP技术吧！

<img src="https://ai-studio-static-online.cdn.bcebos.com/d953727af0c24a7c806ab529495f0904f22f809961be420b8c88cdf59b837394" width="200" height="250" >